# **Assignment 2: Feature Selection**
Sonia Wu and Avital Palchik (Group 26)

Machine Learning for Business Decision Making, Assignment 2

Due December 15th, 23:59



---


## Step 1

To build our final model, we will start with the same dataset and pipeline as Assignment 1. To start, we import all the libraries we will need for this assignment, and set up the seed, file paths, and import the data.

We also have to preprocess in the same way we did for assignment 1:
- For the `pdays` variable, we create a new variable `was_contacted` and replace the -1 values with 0
- We use the Holdout method with the established test set of size 20%

Our best model from Assignment 1 was Gradient Boosting with RandomSearch HPO. For this assignment, we use the same model architecture with default hyperparameters (to be able to analyze feature selection methods).

In [ ]:
# 1. imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score

# 2. setup
STUDENT_ID = 100574955
np.random.seed(STUDENT_ID)

# 3. load data
try:
    df = pd.read_pickle('bank_26.pkl')
    df_comp = pd.read_pickle('bank_competition.pkl')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Files not found. Ensure bank_26.pkl is in the folder.")

# 4. feature engineering
def clean_data(dataframe):
    df_copy = dataframe.copy()
    # create the binary flag
    df_copy['was_contacted'] = (df_copy['pdays'] != -1).astype(int)
    # replace -1 with 0 to fix the scale
    df_copy['pdays'] = df_copy['pdays'].replace(-1, 0)
    return df_copy

df_clean = clean_data(df)

# 5. split X and y
X = df_clean.drop('deposit', axis=1)
y = df_clean['deposit']

# 6. train/test split (0.2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=STUDENT_ID, stratify=y)

print("Data restored and split.")
print(f"Training shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")

Data loaded successfully.
Data restored and split.
Training shape: (8800, 17)
Test shape: (2200, 17)


---
## Step 2
Next, we want to preprocess our data (filling in missing data, scaling numerical features, and converting text categories into numbers). We will create two different pipelines with the two different feature selection methods: one will use `f_classif`, and the other will use `mutual_info_classif`. This will allow us to compare the two methods.

In [ ]:
# 1. redefine the preprocessing
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# define steps
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_cols),
    ('cat', cat_pipeline, categorical_cols)
])

# 2. define the model (Gradient Boosting with default parameters)
gb_model = HistGradientBoostingClassifier(random_state=STUDENT_ID)

# 3. create the two pipelines (f_classif and mutual_info_classif)
# pipeline A: f_classif
pipe_f = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(score_func=f_classif)),
    ('classifier', gb_model)
])

# pipeline B: mutual_info_classif
pipe_mutual = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(score_func=mutual_info_classif)),
    ('classifier', gb_model)
])

---
## Step 3
Now, we will test values of `k` (number of features to be selected) using a grid search. We started with keeping 5 features, and then 10, 15, 20, and so on, up to 'all'. After OneHotEncoding, we should have 53 different features.



In [ ]:
# 1. define the Grid for k
param_grid = {
    'selector__k': [5, 10, 15, 20, 25, 'all']
}

# 2. run the Grid Search to tune k
def run_selection_search(name, pipeline, X_train, y_train):
    print(f"Running Grid Search for {name}...")

    grid = GridSearchCV(
        pipeline,
        param_grid,
        cv=5,                 # we use 5-fold cross-validation
        scoring='accuracy',   # metric is still accuracy
        n_jobs=-1             # we use all CPU cores
    )

    grid.fit(X_train, y_train)

    print(f"  Best Accuracy: {grid.best_score_:.4f}")
    print(f"  Best k (Number of Features): {grid.best_params_['selector__k']}")
    return grid

# 3. run and collect results for both pipelines
results_f = run_selection_search("ANOVA (f_classif)", pipe_f, X_train, y_train)
print("-" * 30)
results_mutual = run_selection_search("Mutual Info", pipe_mutual, X_train, y_train)

Running Grid Search for ANOVA (f_classif)...
  Best Accuracy: 0.8660
  Best k (Number of Features): all
------------------------------
Running Grid Search for Mutual Info...
  Best Accuracy: 0.8660
  Best k (Number of Features): all


---
## Step 4

As we can see, both models perform the same. The best number of features to use is all the features (so all 53). This makes sense because we are using Gradient Boosting, which already eliminates useless features. This implies that explicit feature removal did not improve performance.

This also means both models (`f_classif` and `mutual_info_classif`), in this case, have the same accuracy.

Moving on to the next step, we want to see which features are actually selected and why they are selected. Note that GridSearchCV was performed on the training set only, and the test set was used once for final evaluation.

In [ ]:
# 1. get the feature names
# we need to fit the preprocessor to know the names of the OHE columns
preprocessor.fit(X_train, y_train)
feature_names = preprocessor.get_feature_names_out()

# print which feature names have been selected for each method
selected_f = feature_names[
    results_f.best_estimator_.named_steps['selector'].get_support()
]

selected_m = feature_names[
    results_mutual.best_estimator_.named_steps['selector'].get_support()
]

print("\nSelected features (ANOVA):")
print(selected_f.tolist())

print("\nSelected features (Mutual Info):")
print(selected_m.tolist())


print(f"Total features after One-Hot Encoding: {len(feature_names)}")

# 2. extract scores from the best estimators
# we access the 'selector' step inside the best pipeline found by GridSearch
anova_scores = results_f.best_estimator_.named_steps['selector'].scores_
mutual_scores = results_mutual.best_estimator_.named_steps['selector'].scores_

# 3. create a dataframe to compare rankings
df_scores = pd.DataFrame({
    'Feature': feature_names,
    'ANOVA Score': anova_scores,
    'Mutual Info Score': mutual_scores
})

# 4. show top 10 features by ANOVA
print("\n--- TOP 10 FEATURES (ANOVA) ---")
print(df_scores.sort_values(by='ANOVA Score', ascending=False).head(10))

# 5. show top 10 features by Mutual Info
print("\n--- TOP 10 FEATURES (MUTUAL INFO) ---")
print(df_scores.sort_values(by='Mutual Info Score', ascending=False).head(10))

# 6. final evaluation on test dataset
print("\n--- FINAL TEST EVALUATION ---")

# evaluate ANOVA pipeline
y_pred_f = results_f.predict(X_test)
acc_f = accuracy_score(y_test, y_pred_f)
print(f"Test Accuracy (ANOVA - k={results_f.best_params_['selector__k']}): {acc_f:.4f}")

# evaluate Mutual Info pipeline
y_pred_m = results_mutual.predict(X_test)
acc_m = accuracy_score(y_test, y_pred_m)
print(f"Test Accuracy (Mutual Info - k={results_mutual.best_params_['selector__k']}): {acc_m:.4f}")


Selected features (ANOVA):
['num__age', 'num__balance', 'num__day', 'num__duration', 'num__campaign', 'num__pdays', 'num__previous', 'num__was_contacted', 'cat__job_admin.', 'cat__job_blue-collar', 'cat__job_entrepreneur', 'cat__job_housemaid', 'cat__job_management', 'cat__job_retired', 'cat__job_self-employed', 'cat__job_services', 'cat__job_student', 'cat__job_technician', 'cat__job_unemployed', 'cat__job_unknown', 'cat__marital_divorced', 'cat__marital_married', 'cat__marital_single', 'cat__marital_None', 'cat__education_primary', 'cat__education_secondary', 'cat__education_tertiary', 'cat__education_unknown', 'cat__default_no', 'cat__default_yes', 'cat__housing_no', 'cat__housing_yes', 'cat__loan_no', 'cat__loan_yes', 'cat__contact_cellular', 'cat__contact_telephone', 'cat__contact_unknown', 'cat__month_apr', 'cat__month_aug', 'cat__month_dec', 'cat__month_feb', 'cat__month_jan', 'cat__month_jul', 'cat__month_jun', 'cat__month_mar', 'cat__month_may', 'cat__month_nov', 'cat__month_

---
## Analysis
Even though the grid search revealed that it would be best to use all features for both pipelines, we can still learn from which features held the most weight:
- Top Shared Features: Both methods agreed that `duration` (last contact duration) and `poutcome_success` (previous campaign outcome) were the most critical predictors.
- ANOVA (Linear): Prioritized categorical flags like `contact_unknown` and `marital_None`. It favoured simple "Yes/No" splits.
- Mutual Information (Non-Linear): Prioritized numerical variables like `pdays`, `balance`, and `age`. It would also be able to capture complex, non-linear relationships that ANOVA misses (e.g., the U-shaped relationship where both very young and very old people are more likely to subscribe).

Both SelectKBest methods selected all available features (`k = 53`), indicating that removing features reduced cross-validated accuracy. This is expected when using tree-based models such as Gradient Boosting, which perform implicit feature selection during training. As a result, neither ANOVA nor Mutual Information improved performance compared to Assignment 1.

Our original model remained comparable between the two assignments. Any slight accuracy differences are likely due to switching our parameters to the default settings, rather than the feature selection. To finish off the assignment, we will retrain the model and make our predictions.

In [ ]:
# 1. retrain the winner on FULL data
# since both pipelines chose 'all' features and had the same accuracy,
# we can use the Mutual Info pipeline
# we fit it on X (all training data) and y.
print("Retraining final model on full dataset...")

final_pipeline = results_mutual.best_estimator_
final_pipeline.fit(X, y)

print("Final model retrained successfully.")

# 2. process competition data
# we must apply the 'pdays' fix to the competition data again
print("Processing competition data...")

# apply the manual function we wrote in assignment 1
df_comp_clean = clean_data(df_comp)

# ensure columns match X, make sure we feed the pipeline the same columns as X
X_comp = df_comp_clean[X.columns]

# 3. make predictions
comp_predictions = final_pipeline.predict(X_comp)
print(f"Predictions generated. First 5: {comp_predictions[:5]}")

# 4. save files
# A. the predictions CSV
submission_filename = 'group_26_assignment2_prediction.csv'
pd.DataFrame(comp_predictions, columns=['prediction']).to_csv(submission_filename, index=False)
print(f"Saved predictions to {submission_filename}")

# B. the model pkl file
model_filename = 'group_26_assignment2_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(final_pipeline, file)
print(f"Saved model to {model_filename}")

print("\n--- ASSIGNMENT 2 COMPLETE ---")

Retraining final model on full dataset...
Final model retrained successfully.
Processing competition data...
Predictions generated. First 5: ['no' 'yes' 'no' 'yes' 'yes']
Saved predictions to group_26_assignment2_prediction.csv
Saved model to group_26_assignment2_model.pkl

--- ASSIGNMENT 2 COMPLETE ---
